In [1]:

# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("test") \
      .getOrCreate()


In [3]:
import os
print(os.getcwd())
print(os.listdir(os.getcwd()))

path = os.path.join(os.getcwd(), "Documents","GitHub", "streaming_dwh", "data", "Product")

# print(os.listdir(path))
# print(os.listdir("./Documents/GitHub/streaming_dwh/data/Product/"))

/home/peter
['.devcontainer', '.git', '.gitattributes', '.gitignore', '.idea', '.vscode', 'bronze_products.ipynb', 'data', 'README.md', 'sales_generator', 'test.ipynb']


In [13]:
path = os.path.join(os.getcwd(), "data", "Product", "CS09.json")

df = spark.read.json(path)

# df = spark.read.option("multiline","true").json("./Documents/GitHub/streaming_dwh/data/Product/2024-02-12 21:11:46.767974.json")

df.show()

+-----------------+----+-----------------+--------------+-------------+----------------+--------------------+------------------+-----+----------+------+
|         category|cogs|contains_caffeine|contains_fruit|contains_nuts|contains_veggies|          event_time|              item|price|product_id|  size|
+-----------------+----+-----------------+--------------+-------------+----------------+--------------------+------------------+-----+----------+------+
|Classic Smoothies| 1.5|            false|          true|        false|           false|2024-02-12 21:25:...|Strawberry Limeade| 4.99|      CS09|24 oz.|
+-----------------+----+-----------------+--------------+-------------+----------------+--------------------+------------------+-----+----------+------+



In [19]:

path = os.path.join(os.getcwd(), "data", "Product")

print(os.listdir(path=path))

# To allow automatic schemaInference while reading
# spark.conf.set("spark.sql.streaming.schemaInference", True)

# Create the streaming_df to read from input directory
streaming_df = spark.readStream \
    .format("json") \
    .option("maxFilesPerTrigger", 1) \
    .load(path) \
    .writeStream \
    .format("console") \
    .start()

# streaming_df.display()

['CS01.json', 'CS02.json', 'CS03.json', 'CS04.json', 'CS05.json', 'CS06.json', 'CS07.json', 'CS08.json', 'CS09.json', 'CS10.json', 'CS11.json', 'IS01.json', 'IS02.json', 'IS03.json', 'IS04.json', 'SC01.json', 'SC02.json', 'SC03.json', 'SC04.json', 'SC05.json', 'SF01.json', 'SF02.json', 'SF03.json', 'SF04.json', 'SF05.json', 'SF06.json', 'SF07.json']


In [8]:
import tempfile
import time
with tempfile.TemporaryDirectory() as d:
    # Write a temporary JSON file to read it.
    spark.createDataFrame(
        [(100, "Hyukjin Kwon"),], ["age", "name"]
    ).write.mode("overwrite").format("json").save(d)

    # Start a streaming query to read the JSON file.
    q = spark.readStream.schema(
        "age INT, name STRING"
    ).json(d).writeStream.format("console").start()
    time.sleep(3)
    q.stop()